# Carga de Librerías, datos y funciones:

Fuente: 
https://kgptalkie.com/sentiment-classification-using-bert/

In [1]:
#!pip install ktrain

In [2]:
import pickle
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
import re
import string
import spacy

from string import digits

sp = spacy.load('en_core_web_sm')

In [3]:
def cut_text_len(list):
    temp = []
    temp_coma = []
    temp_list = []

    for sentence in list: temp.extend(sentence.split('.'))
    for sentence in temp: temp_coma.extend(sentence.split(','))
    temp_coma = [sentence for sentence in temp_coma if len(sentence.split()) > 2]

    for sentence in temp_coma:
        text = sentence.split()
        new_sentence = ''
        len_sentence = 0
        mess = len(text)
        while mess > 0:
            new_sentence += text[0] + ' '
            text.pop(0)
            len_sentence += 1
            mess = len(text)
            if len_sentence == 15:            
                temp_list.append(new_sentence)
                new_sentence = ''
                len_sentence = 0
                mess = len(text)
            elif mess == 0:
                mess = len(new_sentence.split())
                if mess <= 2:
                    break
                temp_list.append(new_sentence)
                break
    return(temp_list)

def clean_text(list):   
    text_clean = []
    for text in list:
        text = text.lower()
        new_text = ''
        for word in text.split():
            for key in contractions:
                if word == key:
                    word = contractions[key]
            new_text += word + ' '
        text = re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', new_text) 
        text = re.sub(patron_web, '', text)
        table = str.maketrans(' ', ' ', string.punctuation)
        stripped = [word.translate(table) for word in text]
        text = ''.join(stripped)
        table = str.maketrans('', '', digits)
        newtext = text.translate(table)
        newtext = re.sub("\'", '', newtext)
        text_clean.append(newtext)

    return(text_clean)

# Fuente: https://kgptalkie.com/3664-2/
# Lista de contracciones en inglés extraída de "NLP: End to End Text Processing for Beginners" (., Roshan)

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"aint": "am not",
"arent": "are not",
"cant": "can not",
"cantve": "cannot have",
"cause": "because",
"couldve": "could have",
"couldnt": "could not",
"couldntve": "could not have",
"didnt": "did not",
"doesnt": "does not",
"dont": "do not",
"hadnt": "had not",
"hadntve": "had not have",
"hasnt": "has not",
"havent": "have not",
"hed": "he would",
"hedve": "he would have",
"hell": "he will",
"hellve": "he will have",
"hes": "he is",
"howd": "how did",
"howdy": "how do you",
"howll": "how will",
"hows": "how does",
"id": "i would",
"idve": "i would have",
"ill": "i will",
"illve": "i will have",
"im": "i am",
"ive": "i have",
"isnt": "is not",
"itd": "it would",
"itdve": "it would have",
"itll": "it will",
"itllve": "it will have",
"its": "it is",
"lets": "let us",
"maam": "madam",
"maynt": "may not",
"mightve": "might have",
"mightnt": "might not",
"mightntve": "might not have",
"mustve": "must have",
"mustnt": "must not",
"mustntve": "must not have",
"neednt": "need not",
"needntve": "need not have",
"oclock": "of the clock",
"oughtnt": "ought not",
"oughtntve": "ought not have",
"shant": "shall not",
"shant": "shall not",
"shantve": "shall not have",
"shed": "she would",
"shedve": "she would have",
"shell": "she will",
"shellve": "she will have",
"shes": "she is",
"shouldve": "should have",
"shouldnt": "should not",
"shouldntve": "should not have",
"sove": "so have",
"sos": "so is",
"thatd": "that would",
"thatdve": "that would have",
"thats": "that is",
"thered": "there would",
"theredve": "there would have",
"theres": "there is",
"theyd": "they would",
"theydve": "they would have",
"theyll": "they will",
"theyllve": "they will have",
"theyre": "they are",
"theyve": "they have",
"tove": "to have",
"wasnt": "was not", 
"u": " you  ",
"ur": "your", 
"n": "and", 
"friends": "",
"friend":"",
"people":"",
"girl":"",
"girlfriend": "",
"grl":"",
"nt": "not",
"gf": "", 
"cuttersuicidal": "suicidal", 
"idk": "i do not know", 
"theripist": "therapist",
"s": "is",
"mt": "",
"cuttersuicidal": "cutter suicidal"
}

patron_web = r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

def duplicated(list): 
    temp = []
    for sentence in list:
        new_sentence = sentence.split() 
        new_sentece_join = ' '.join(new_sentence)
        temp.append(new_sentece_join)
    return(temp)

def text_to_spacy(list): 
    temp = []
    for sentence in list:
        new_sentence = ''
        for word in sp(str(sentence)):
            new_sentence += word.lemma_ + ' '
        temp.append(new_sentence.lower())
    return(temp)

In [5]:

df = pickle.load(open("./data/df.p", "rb" ))

In [6]:
df.head()

,text,label,label_bi,index
651,ex have onset and anxiety,flagged,1,-909
108,dumps expressing one schoolwork so think my li...,not_flagged,0,-903
441,that need it be a fairly common occurrence,not_flagged,0,-837
736,i call back and get a hospital number so i kno...,flagged,1,-833
145,gave least went campingsurfing week work throu...,not_flagged,0,-783


In [7]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

In [8]:
print("Size of train dataset: ", train.shape)
print("Size of test dataset: ", test.shape)

Size of train dataset:  (822, 4)
Size of test dataset:  (186, 4)


In [9]:
train

,text,label,label_bi,index
651,ex have onset and anxiety,flagged,1,-909
108,dumps expressing one schoolwork so think my li...,not_flagged,0,-903
736,i call back and get a hospital number so i kno...,flagged,1,-833
145,gave least went campingsurfing week work throu...,not_flagged,0,-783
321,others even truth hurts best way start even would,not_flagged,0,-758
...,...,...,...,...
751,i try to be there for she when she need it and...,flagged,1,885
482,try kind helpful honest,not_flagged,0,897
760,issue suicide every day grade kindness of a ho...,flagged,1,948
860,people person he be not the throughout to,flagged,1,976


# Preparación de la data y modelo

In [11]:
# El framework de K-train tiene un método para la separación de las cadenas 
## la función que preprocesará la data.

(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df= train,
                                                                   text_column = 'text',
                                                                   label_columns = 'label',
                                                                   val_df = test,
                                                                   maxlen = 16,
                                                                   preprocess_mode = 'bert')

['flagged', 'not_flagged']
     flagged  not_flagged
651      1.0          0.0
108      0.0          1.0
736      1.0          0.0
145      0.0          1.0
321      0.0          1.0
['flagged', 'not_flagged']
     flagged  not_flagged
441      0.0          1.0
451      0.0          1.0
753      1.0          0.0
78       0.0          1.0
550      1.0          0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [16]:
X_train[0].shape

(822, 16)

In [13]:
# Creación del modelo

model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 16
done.


In [14]:
# Compilación del modelo

learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 6)

In [15]:
#Proceso de entrenamiento

learner.fit_onecycle(lr = 2e-5, epochs = 5)
predictor = ktrain.get_predictor(learner.model, preproc)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
137/137 [==============================] - 201s 1s/step - loss: 0.6971 - accuracy: 0.5864 - val_loss: 0.6238 - val_accuracy: 0.6774
Epoch 2/5
137/137 [==============================] - 164s 1s/step - loss: 0.4323 - accuracy: 0.8078 - val_loss: 0.3627 - val_accuracy: 0.8441
Epoch 3/5
137/137 [==============================] - 161s 1s/step - loss: 0.1569 - accuracy: 0.9331 - val_loss: 0.3521 - val_accuracy: 0.9086
Epoch 4/5
137/137 [==============================] - 173s 1s/step - loss: 0.0268 - accuracy: 0.9927 - val_loss: 0.3573 - val_accuracy: 0.9032
Epoch 5/5
137/137 [==============================] - 156s 1s/step - loss: 0.0132 - accuracy: 0.9964 - val_loss: 0.3412 - val_accuracy: 0.9247


In [17]:
predictor = ktrain.get_predictor(learner.model, preproc)

# Ejemplo #1

In [26]:
new_sentences = ["I've some anxiety and some days want to die", 
                        "I help her with her depression",
                        "I'm very sad and depressed today. want to finish it ", 
                        "Some of my friends are suicidal", 
                        "I am happy and joyful "]
                        

In [28]:
predictor.predict(new_sentences)


['flagged', 'flagged', 'not_flagged', 'flagged', 'not_flagged']

In [29]:
new_sentences = cut_text_len(new_sentences)
new_sentences = clean_text(new_sentences)
new_sentences = text_to_spacy(new_sentences)
new_sentences = duplicated(new_sentences)

In [30]:
predictor.predict(new_sentences)


['flagged', 'flagged', 'flagged', 'flagged', 'flagged', 'flagged']

# Ejemplo #2

In [38]:

new_sentences = ["my name is andie. i am 16 years old. my life is too messed up for fixing. i have been through a lot. i think it's my time to go. there's no other way . i will not be stating my method because i don't want to be copied. i love you, stranger, please live a better life than me."]

In [33]:
new_sentences = cut_text_len(new_sentences)
new_sentences = clean_text(new_sentences)
new_sentences = text_to_spacy(new_sentences)
new_sentences = duplicated(new_sentences)

In [39]:
predictor.predict(sentences_full)

['not_flagged']

In [37]:
i = 0
for sentence in new_sentences: 
    print(sentence , ':',  str(total[i]))
    i += 1

my name be andie : flagged
i be year old : not_flagged
my life be too mess up for fix : not_flagged
i have be through a lot : not_flagged
i think it be my time to go : not_flagged
there be no other way : not_flagged
i will not be state my method because i do not want to be copy : not_flagged
i love you : flagged
please live a well life than i : flagged


# Ejemplo #3

In [40]:
new_sentences = ["Can someone explain to me how actively looking for help is one of the loneliest experiences someone can have? Not only am I miserable because I'm sick, but then I also have to listen to these people who are supposed to help me that I have to want to get better or it won't work. Now, I grew up evangelical (not practicing, agnostic, forever put off by religion) and I sure as hell know that expression. What if I don't believe in getting better. What if a big part of me just wants to decay. What if I dragged myself to your office with the last bit of strength in my body. If it were for me, I'd just lie down and die. Deep down, I don't want to get better, I want a do over. Everything is fucked. There's no ladder out of this hole, I only have a shovel and I'm going to dig myself a grave."]

In [42]:
predictor.predict(new_sentences)

['not_flagged']